# Financial Model Prediction

In [1]:
import pandas as pd
import numpy as np
import seaborn as sbn
import re

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

import lightgbm as lgb

%matplotlib inline

C:\Users\esteb\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
def add_shifts(df_, col_to_shift, new_col, shift):
    """
    This function add shifted columns to data by ticker.
    
    :param pd.DataFrame df_: Dataframe with financial data.
    :param str col_to_shift: Column over to create the shift.
    :param str new_col: Name of the shifted column.
    :param int shift: Days to use as shift.
    :return pd.DataFrame: Dataframe with the shift added.
    """
    
    for id_ in df_['ticker'].unique():
        df_by_id = df_[df_['ticker'] == id_]
        df_.loc[df_['ticker'] == id_, new_col] = df_by_id[col_to_shift] - df_by_id[col_to_shift].shift(shift)
        
    return df_

In [7]:
def get_non_n_cols(df_, n):
    """
    Get the columns that his window is not n days
    
    :param pd.DataFrame df_: Dataframe with financial data.
    :param int n: n days to get columns with.
    :return list: List with the name of the columns.
    """
    
    return [elem for elem in df_.columns if (re.search(r'\d+$', elem) is not None) and (int(elem[-2:].strip()) != n)]

In [8]:
def interpolate_nan_values(df_, to_interpolate):
    """
    Interpolate and extrapolate nan values for numerical columns.
    
    :param pd.DataFrame df_: Dataframe with financial data with NaN values.
    :param list to_interpolate: List with columns to interpolate.
    :return pd.DataFrame: Dataframe with financial data without NaN values.
    """
    
    list_df = []
    for tick in df_['ticker'].unique():
        df_by_ticker = df_[df_['ticker'] == tick]
        for col in to_interpolate:
            df_by_ticker[col] = df_by_ticker[col].interpolate(method='linear', limit_direction='both')
        list_df.append(df_by_ticker)
    return pd.concat(list_df)

In [9]:
def categorize_each_difference(num_list, df_):
    """
    This function categorize the shifted columns in Weak Bull o Bear (W. Bull, W. Bear), 
    Bull or Bear and Strong Bull or Bear (S. Bull, S. Bear) depending on the value of the shifted column and
    his statistics (median, p25, p75) by ticker, year, month and sign.
    
    :param list num_list: List with days to categorize.
    :param pd.DataFrame df_: Dataframe to categorize.
    :return pd.DataFrame: Dataframe recalculated.
    """
    cols_to_keep = list(df_.columns)
    df_['year'], df_['month'] = df_['date'].dt.year, df_['date'].dt.month
    for num_ in num_list:
        df_.loc[df_['close_shifted_%i' % num_] >= 0, 'sign_%i' % num_] = 'Bull'
        df_.loc[df_['close_shifted_%i' % num_] < 0, 'sign_%i' % num_] = 'Bear'
        group = df_.groupby(['ticker', 'year', 'month', 'sign_%i' % num_])['close_shifted_%i' % num_].describe()
        group = group[['25%', '50%', '75%', 'std']].reset_index()
        group.rename({'std': 'std_%i' % num_}, axis='columns', inplace=True)
        df_ = pd.merge(left=df_, right=group, on=['ticker', 'year', 'month', 'sign_%i' % num_], how='inner')
        
        df_.loc[(df_['sign_%i' % num_] == 'Bull') & 
                (df_['close_shifted_%i' % num_] <= df_['50%']), 'cat_close_shifted_%i' % num_] = 'W. ' + df_['sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bull') & 
                   (df_['close_shifted_%i' % num_] > df_['50%']) &
                   (df_['close_shifted_%i' % num_] < df_['75%']), 'cat_close_shifted_%i' % num_] = df_['sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bull') &
                   (df_['close_shifted_%i' % num_] >= df_['75%']), 'cat_close_shifted_%i' % num_] = 'S. ' + df_['sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bear') & 
                (df_['close_shifted_%i' % num_] >= df_['50%']), 'cat_close_shifted_%i' % num_] = 'W. ' + df_['sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bear') & 
                   (df_['close_shifted_%i' % num_] < df_['50%']) &
                   (df_['close_shifted_%i' % num_] > df_['25%']), 'cat_close_shifted_%i' % num_] = df_['sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bear') & 
                   (df_['close_shifted_%i' % num_] <= df_['25%']), 'cat_close_shifted_%i' % num_] = 'S. ' + df_['sign_%i' % num_]
        
        df_.drop(['25%', '50%', '75%'], axis='columns', inplace=True)
        cols_to_keep.extend(['cat_close_shifted_%i' % num_, 'std_%i' % num_])
    return df_[cols_to_keep]

In [10]:
df_categorical = pd.read_csv('../data/db_bsm_categorical.csv')
df_financial = pd.read_csv('../data/db_bsm_financial.csv')

In [11]:
df_financial.replace(0, np.NaN, inplace=True)
df_financial.isnull().sum().to_frame('Null Values').loc[['close', 'volume']]

,Null Values
close,1081
volume,1996


In [15]:
df_financial_not_nan = interpolate_nan_values(df_financial, ['close', 'volume'])

In [16]:
df_financial_not_nan.isnull().sum().to_frame('Null Values').loc[['close', 'volume']]

,Null Values
close,0
volume,0


In [17]:
num_list = [3, 5, 7, 14, 21]

In [18]:
for num_ in num_list:
    df_fin = add_shifts(df_financial_not_nan, 'close', 'close_shifted_%i' % num_, num_)

In [19]:
df_fin.dropna(subset=['close_shifted_21'], inplace=True)

In [20]:
df_fin_not_nan = interpolate_nan_values(df_fin, list(df_fin.select_dtypes(float)))

In [21]:
df_fin_not_nan.isnull().sum().to_frame('Null Values').sort_values(by='Null Values', ascending=False).head(3)

,Null Values
ADX 14,0
RSI 21,0
ROCR 14,0


In [22]:
df_fin_not_nan['date'] = pd.to_datetime(df_fin_not_nan['date'])

In [23]:
%time df_final = categorize_each_difference(num_list, df_fin_not_nan)

Wall time: 4min 50s


In [24]:
df_categorical = df_categorical.dropna()
df_categorical = df_categorical.drop_duplicates(subset=['ticker'], keep='first')

In [25]:
df_final = pd.merge(left=df_final, right=df_categorical, how='inner', on='ticker')

In [26]:
df_final['sector_gics'].unique()

array(['Financials', 'Consumer discretionary', 'Communication services',
       'Energy', 'Industrials', 'Healthcare', 'Information technology',
       'Consumer staples', 'Utilities'], dtype=object)

In [27]:
# %time df_final = pd.get_dummies(df_final, columns='sector_gics', prefix='sector_gics_')

In [28]:
df_final.loc[df_final['sector_gics'] == 'Utilities', 'sector_gics_Utilities'] = 1
df_final.loc[df_final['sector_gics'] == 'Healthcare', 'sector_gics_Heatlhcare'] = 1
df_final.loc[df_final['sector_gics'] == 'Financials', 'sector_gics_Financials'] = 1
df_final.loc[df_final['sector_gics'] == 'Consumer discretionary', 'sector_gics_Consumer_discretionary'] = 1
df_final.loc[df_final['sector_gics'] == 'Communication services', 'sector_gics_Communication_services'] = 1
df_final.loc[df_final['sector_gics'] == 'Energy', 'sector_gics_Energy'] = 1
df_final.loc[df_final['sector_gics'] == 'Industrials', 'sector_gics_Industrials'] = 1
df_final.loc[df_final['sector_gics'] == 'Information technology', 'sector_gics_Information_technology'] = 1
df_final.loc[df_final['sector_gics'] == 'Consumer staples', 'sector_gics_Consumer_staples'] = 1

In [29]:
df_final.columns

Index(['ADX 14', 'ADX 21', 'ADX 3', 'ADX 42', 'ADX 5', 'ADX 7',
       'Aroon Down 14', 'Aroon Down 21', 'Aroon Down 3', 'Aroon Down 42',
       ...
       'country', 'sector_gics_Utilities', 'sector_gics_Heatlhcare',
       'sector_gics_Financials', 'sector_gics_Consumer_discretionary',
       'sector_gics_Communication_services', 'sector_gics_Energy',
       'sector_gics_Industrials', 'sector_gics_Information_technology',
       'sector_gics_Consumer_staples'],
      dtype='object', length=177)

In [30]:
df_final[['sector_gics_Utilities', 'sector_gics_Heatlhcare',
          'sector_gics_Financials', 'sector_gics_Consumer_discretionary',
          'sector_gics_Communication_services', 'sector_gics_Energy',
          'sector_gics_Industrials', 'sector_gics_Information_technology',
          'sector_gics_Consumer_staples']] = df_final[['sector_gics_Utilities', 'sector_gics_Heatlhcare',
          'sector_gics_Financials', 'sector_gics_Consumer_discretionary',
          'sector_gics_Communication_services', 'sector_gics_Energy',
          'sector_gics_Industrials', 'sector_gics_Information_technology',
          'sector_gics_Consumer_staples']].replace(np.NaN, 0)

In [31]:
df_final.head()

,ADX 14,ADX 21,ADX 3,ADX 42,ADX 5,ADX 7,Aroon Down 14,Aroon Down 21,Aroon Down 3,Aroon Down 42,Aroon Down 5,Aroon Down 7,Aroon Up 14,Aroon Up 21,Aroon Up 3,Aroon Up 42,Aroon Up 5,Aroon Up 7,BOP 14,BOP 21,BOP 3,BOP 42,BOP 5,BOP 7,CCI 14,CCI 21,CCI 3,CCI 42,CCI 5,CCI 7,CMO 14,CMO 21,CMO 3,CMO 42,CMO 5,CMO 7,Chaikin A/D,DEMA 14,DEMA 21,DEMA 3,DEMA 42,DEMA 5,DEMA 7,DX 14,DX 21,DX 3,DX 42,DX 5,DX 7,FAMA,KAMA 14,KAMA 21,KAMA 3,KAMA 42,KAMA 5,KAMA 7,MACD,MACD_Hist,MACD_Signal,MAMA,MINUS_DI 14,MINUS_DI 21,MINUS_DI 3,MINUS_DI 42,MINUS_DI 5,MINUS_DI 7,MINUS_DM 14,MINUS_DM 21,MINUS_DM 3,MINUS_DM 42,MINUS_DM 5,MINUS_DM 7,MOM 14,MOM 21,MOM 3,MOM 42,MOM 5,MOM 7,NATR 14,NATR 21,NATR 3,NATR 42,NATR 5,NATR 7,OBV,PLUS_DI 14,PLUS_DI 21,PLUS_DI 3,PLUS_DI 42,PLUS_DI 5,PLUS_DI 7,PLUS_DM 14,PLUS_DM 21,PLUS_DM 3,PLUS_DM 42,PLUS_DM 5,PLUS_DM 7,PPO,ROCR 14,ROCR 21,ROCR 3,ROCR 42,ROCR 5,ROCR 7,RSI 14,RSI 21,RSI 3,RSI 42,RSI 5,RSI 7,Real Lower Band 14,Real Lower Band 21,Real Lower Band 3,Real Lower Band 42,Real Lower Band 5,Real Lower Band 7,Real Middle Band 14,Real Middle Band 21,Real Middle Band 3,Real Middle Band 42,Real Middle Band 5,Real Middle Band 7,Real Upper Band 14,Real Upper Band 21,Real Upper Band 3,Real Upper Band 42,Real Upper Band 5,Real Upper Band 7,SAR,SMA 14,SMA 21,SMA 3,SMA 42,SMA 5,SMA 7,SlowD,SlowK,ULTOSC,WMA 14,WMA 21,WMA 3,WMA 42,WMA 5,WMA 7,close,date,ticker,volume,close_shifted_3,close_shifted_5,close_shifted_7,close_shifted_14,close_shifted_21,cat_close_shifted_3,std_3,cat_close_shifted_5,std_5,cat_close_shifted_7,std_7,cat_close_shifted_14,std_14,cat_close_shifted_21,std_21,company,sector_gics,sector_icb,stock_index,country,sector_gics_Utilities,sector_gics_Heatlhcare,sector_gics_Financials,sector_gics_Consumer_discretionary,sector_gics_Communication_services,sector_gics_Energy,sector_gics_Industrials,sector_gics_Information_technology,sector_gics_Consumer_staples
0,29.4543,23.4455,50.8273,12.4128,38.1828,35.1222,21.4286,47.6190,100.0000,11.9048,100.0,100.00000,78.57140,85.7143,100.0000,92.8571,40.0,57.1429,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-72.1630,-58.3951,-100.0000,41.0458,-136.0759,-172.4664,-8.6086,2.3899,-74.5495,9.2424,-50.2514,-35.1149,1.160891e+08,50.7378,50.8875,49.7894,50.8542,50.1555,50.3899,5.7901,12.2554,57.5078,13.5135,30.3172,15.6000,43.9790,50.1341,50.1659,50.2751,49.7966,50.3814,50.4775,0.4918,-0.1192,0.6110,45.9606,19.1776,17.5504,38.7945,17.3277,30.2990,25.5176,2.2749,3.1482,1.1767,6.8357,1.4047,1.5925,-0.49480,0.0792,-1.6824,1.3855,-1.6824,-0.79170,1.7124,1.7263,2.0433,1.8982,1.8738,1.8017,145958362.0,21.5349,22.4529,10.4659,22.7426,16.2014,18.6305,2.5546,4.0276,0.3175,8.9718,0.7511,1.1627,0.9535,0.9901,1.0016,0.9671,1.0288,0.9671,0.9843,45.6957,51.1950,12.7252,54.6212,24.8743,32.4426,49.2264,49.2035,48.9728,47.2615,49.3927,49.6085,50.3874,50.2555,50.4393,49.3464,50.6900,50.7550,51.5485,51.3075,51.9057,51.4313,51.9873,51.9015,51.8578,50.3874,50.2555,50.4393,49.3464,50.6900,50.7550,49.8208,29.2586,45.5067,50.5411,50.4161,50.1424,49.8882,50.4525,50.5960,49.4826,2004-01-30,BNP.PA,3328440.0,-1.6824,-1.6824,-0.7917,-0.4948,0.0792,S. Bear,NaN,S. Bear,NaN,S. Bear,NaN,S. Bear,NaN,S. Bull,NaN,BNP Paribas SA,Financials,Financials,CAC 40,France,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27.9092,22.9722,48.3121,12.4521,35.1961,31.6695,14.2857,42.8571,66.6667,9.5238,80.0,85.71430,71.42860,80.9524,100.0000,90.4762,20.0,42.8571,0.6000,0.6000,0.6000,0.6000,0.6000,0.6000,6.7028,26.9140,37.8375,77.3341,-31.6978,-50.6524,13.4915,16.5795,11.2225,15.4646,6.4748,7.4147,1.178405e+08,50.7643,50.9131,50.3549,50.9074,50.3874,50.4984,7.8230,13.5071,43.2819,14.0642,23.2493,10.9534,43.9892,50.1449,50.1758,50.2886,49.8221,50.3855,50.4783,0.4757,-0.1082,0.5839,46.0077,17.0349,16.2338,23.0211,16.7259,22.0586,20.2680,2.1124,2.9983,0.7845,6.6729,1.1237,1.3650,0.79170,0.9897,-0.5938,2.3752,-0.2969,-0.64325,1.7481,1.7357,2.2417,1.8747,2.0108,1.8988,148877031.0,19.9264,21.3041,9.1129,22.2007,13.7365,16.2662,2.4710,3.9348,0.3105,8.8571

In [32]:
df_final_train = df_final[df_final['date'].dt.year < 2019]
df_final_test = df_final[df_final['date'].dt.year >= 2019]

# 3 Days prediction

In [13]:
target_train = df_final_train['cat_close_shifted_3']
features_train = df_final_train[df_final_train.select_dtypes(float).columns]

In [14]:
knn = GridSearchCV(KNeighborsClassifier(),
                   param_grid={'n_neighbors': range(4, 12)}, 
                   scoring='recall_macro', 
                   cv=3)
knn.fit(features_train.values, target_train.values)
knn = KNeighborsClassifier(**knn.best_params_)
knn.fit(features_train.values, target_train.values)
print(classification_report(y_test, knn.predict(X_test)))

ADX 14                 float64
ADX 21                 float64
ADX 3                  float64
ADX 42                 float64
ADX 5                  float64
ADX 7                  float64
Aroon Down 14          float64
Aroon Down 21          float64
Aroon Down 3           float64
Aroon Down 42          float64
Aroon Down 5           float64
Aroon Down 7           float64
Aroon Up 14            float64
Aroon Up 21            float64
Aroon Up 3             float64
Aroon Up 42            float64
Aroon Up 5             float64
Aroon Up 7             float64
BOP 14                 float64
BOP 21                 float64
BOP 3                  float64
BOP 42                 float64
BOP 5                  float64
BOP 7                  float64
CCI 14                 float64
CCI 21                 float64
CCI 3                  float64
CCI 42                 float64
CCI 5                  float64
CCI 7                  float64
CMO 14                 float64
CMO 21                 float64
CMO 3   